# Neural Network Model

In [1]:
import numpy as np
import os
import torch
import plotly.graph_objects as go

from models.neural_network import NeuralNetworkModel

DATA_PATH = 'raw_data/full_data/'

TOTAL_RECORDINGS = 10
RECORDING_IDS = np.arange(TOTAL_RECORDINGS)
# RECORDING_IDS = [5, 6, 8]
TRAIN_SPLIT = 0.8
NN_WIDTH = 1000
NN_HIDDEN_LAYERS = 3
TRAINING_ITERATIONS = 2001
RANK = 3

In [2]:
files = os.listdir(DATA_PATH)
X_files = [f for f in files if f.endswith('.npy') and f.startswith('X')]
X_files.sort()
Y_files = [f for f in files if f.endswith('.npy') and f.startswith('Y')]
Y_files.sort()

X_files = np.array(X_files)
Y_files = np.array(Y_files)

X_list = [np.load(DATA_PATH + f) for f in X_files[RECORDING_IDS]]
Y_list = [np.load(DATA_PATH + f) for f in Y_files[RECORDING_IDS]]

In [3]:
print(len(X_list))
print(X_list[0].shape)

10
(775, 25, 40)


In [4]:
for i in range(TOTAL_RECORDINGS):
    indices = np.arange(X_list[i].shape[0])
    np.random.shuffle(indices)
    X_list[i] = X_list[i][indices]
    Y_list[i] = Y_list[i][indices]

X_train = [X[:int(TRAIN_SPLIT * X.shape[0])] for X in X_list]
X_test = [X[int(TRAIN_SPLIT * X.shape[0]):] for X in X_list]
Y_train = [Y[:int(TRAIN_SPLIT * Y.shape[0])] for Y in Y_list]
Y_test = [Y[int(TRAIN_SPLIT * Y.shape[0]):] for Y in Y_list]

In [5]:
print(len(X_train))
print(X_train[0].shape)

10
(620, 25, 40)


# Training

In [6]:
def train_step(recording, X_train_recording, Y_train_recording, model, optimizer, loss_fn):
    optimizer.zero_grad()
    
    X_train_recording = torch.from_numpy(X_train_recording).float()
    Y_train_recording = torch.from_numpy(Y_train_recording).float()

    Y_pred = model(recording, X_train_recording)
    
    loss = loss_fn(Y_pred, Y_train_recording)
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [7]:
def evaluate_recording(recording, X_test_recording, Y_test_recording, model, optimizer, loss_fn, plot=False):
    # Test the model
    model.eval()
    
    X_test_recording = torch.from_numpy(X_test_recording).float()
    Y_test_recording = torch.from_numpy(Y_test_recording).float()
    Y_pred = model(recording, X_test_recording)
    loss = loss_fn(Y_pred, Y_test_recording)

    if plot:
        for t in range(6):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=Y_test_recording[t], name='Actual'))
            fig.add_trace(go.Scatter(y=Y_pred[t].detach().numpy(), name='Predicted'))
            fig.update_layout(title=f'Actual and Predicted Wheel Speeds, Recording {recording}, Trial {t}', xaxis_title='Time', yaxis_title='Value')
            fig.show()

    return loss.item()

In [8]:
neural_network_model = NeuralNetworkModel(
    TOTAL_RECORDINGS,
    [X_train[i].shape[1] for i in range(TOTAL_RECORDINGS)], 
    40, 
    NN_WIDTH,
    NN_HIDDEN_LAYERS,
    RANK
)

optimizer = torch.optim.Adam(neural_network_model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

train_loss = []
test_loss = []

for training_iteration in range(TRAINING_ITERATIONS):
    # Get a random permutation of numbers from 0 to TOTAL_RECORDINGS
    # session_order = np.arange(TOTAL_RECORDINGS)
    # np.random.shuffle(session_order)

    train_loss_iter = [recordings for recordings in range(TOTAL_RECORDINGS)]
    test_loss_iter = [recordings for recordings in range(TOTAL_RECORDINGS)]

    for recording in range(TOTAL_RECORDINGS):
        train_loss_item = train_step(recording, X_train[recording], Y_train[recording], neural_network_model, optimizer, loss_fn)
        test_loss_item = evaluate_recording(recording, X_test[recording], Y_test[recording], neural_network_model, optimizer, loss_fn, plot=False)
        
        train_loss_iter[recording] = train_loss_item
        test_loss_iter[recording] = test_loss_item
        
        if training_iteration % 100 == 0:
            print(f'Iteration {training_iteration}, Recording {recording}, Train Loss {train_loss_item}, Test Loss {test_loss_item}')
    
    train_loss.append(train_loss_iter)
    test_loss.append(test_loss_iter)

Iteration 0, Recording 0, Train Loss 1.2109935283660889, Test Loss 0.8486176133155823
Iteration 0, Recording 1, Train Loss 0.7879940271377563, Test Loss 1.3808765411376953


Iteration 0, Recording 2, Train Loss 1.4758191108703613, Test Loss 1.437142014503479
Iteration 0, Recording 3, Train Loss 55.84429168701172, Test Loss 9.902843475341797
Iteration 0, Recording 4, Train Loss 0.9730369448661804, Test Loss 1.3466124534606934
Iteration 0, Recording 5, Train Loss 7.886160373687744, Test Loss 3.3133046627044678
Iteration 0, Recording 6, Train Loss 10.229597091674805, Test Loss 1.3796088695526123
Iteration 0, Recording 7, Train Loss 1.0891125202178955, Test Loss 2.5672693252563477
Iteration 0, Recording 8, Train Loss 0.7640231847763062, Test Loss 1.1056841611862183
Iteration 0, Recording 9, Train Loss 59.5726318359375, Test Loss 6.592728614807129
Iteration 100, Recording 0, Train Loss 0.6366593837738037, Test Loss 0.7119029760360718
Iteration 100, Recording 1, Train Loss 0.7088281512260437, Test Loss 0.731303870677948
Iteration 100, Recording 2, Train Loss 0.6437668204307556, Test Loss 0.6208794116973877
Iteration 100, Recording 3, Train Loss 0.813519418239593

KeyboardInterrupt: 

In [ ]:
train_loss = np.array(train_loss)
test_loss = np.array(test_loss)

fig = go.Figure()
for recording in range(TOTAL_RECORDINGS):
    fig.add_trace(go.Scatter(y=test_loss[:, recording], name=f'Test Recording {recording}'))
    fig.add_trace(go.Scatter(y=train_loss[:, recording], name=f'Train Recording {recording}'))
fig.update_layout(title=f'Loss Curve Rank {RANK}', xaxis_title='Iteration', yaxis_title='Loss')
fig.show()

# Evaluation

In [ ]:
# Print all of the model parameters
# for param in reduced_rank_model.parameters():
#     print(param.data.shape)
#     print(param.data)

# print(f"=== U matrices, length: {len(reduced_rank_model.Us)} ===")
# print(reduced_rank_model.Us)
# print(f"=== V matrices, shape: {reduced_rank_model.V.shape} ===")
# print(reduced_rank_model.V)
# print(f"=== bias, shape: {reduced_rank_model.bias.shape} ===")
# print(reduced_rank_model.bias)

# print(reduced_rank_model.parameters())


=== U matrices, length: 3 ===
[Parameter containing:
tensor([[ 2.0658e-02, -3.3064e-01,  1.5595e-01],
        [ 2.7800e+00, -7.8728e-01, -1.2240e+00],
        [ 1.8036e+00, -7.0499e-01, -1.0640e+00],
        [-1.1790e+00,  1.2190e+00, -4.0396e-01],
        [-1.1465e+00,  6.3781e-01,  6.2859e-01],
        [ 3.9347e-01,  1.1051e-01,  1.5213e+00],
        [ 7.7325e-01, -3.5576e-01, -1.3574e+00],
        [-3.4828e-02, -7.6026e-01,  6.5053e-01],
        [-1.1131e-01,  6.7857e-01,  1.9686e+00],
        [ 1.0294e+00, -8.5254e-01, -2.1510e+00],
        [ 6.0210e-01,  2.9772e-01,  1.1659e+00],
        [-1.0526e+00,  6.6373e-01,  8.1934e-02],
        [-7.5289e-01, -5.5953e-01,  4.3701e-01],
        [-9.8678e-01, -2.3025e-01, -3.9757e-01],
        [ 3.6819e-01,  3.2836e-01,  1.4737e+00],
        [-1.1289e+00, -2.5887e+00, -1.1251e+00],
        [ 6.6701e-01, -1.6614e-01,  5.1608e-02],
        [ 4.9930e-01, -1.3992e-01, -6.8341e-01],
        [-2.1775e+00,  1.1925e-01,  2.1473e+00],
        [-5.7993

In [ ]:
for recording in range(TOTAL_RECORDINGS):
    loss_item = evaluate_recording(recording, X_test[recording], Y_test[recording], neural_network_model, optimizer, loss_fn, plot=True)
    print(f'Evaluation, Recording {recording}, Loss {loss_item}')

Evaluation, Recording 0, Loss 0.9562953114509583


Evaluation, Recording 1, Loss 1.1012259721755981


Evaluation, Recording 2, Loss 0.587719202041626


Evaluation, Recording 3, Loss 3.3642125129699707


Evaluation, Recording 4, Loss 0.5719311833381653


Evaluation, Recording 5, Loss 1.4697034358978271


Evaluation, Recording 6, Loss 1.5042539834976196


Evaluation, Recording 7, Loss 0.7002268433570862


Evaluation, Recording 8, Loss 0.5279169082641602


Evaluation, Recording 9, Loss 1.1715419292449951
